$ max{\,8.1x_1\, + \,10.8x_2} $
$s.t.$
$0.8$

In [1087]:
from pyomo.environ import *

class OptiModel:
    
    def __init__(self):
        self.m = ConcreteModel()
        self.m.dual = Suffix(direction=Suffix.IMPORT)
        self.m.theta1 = Var(initialize=0.0, domain=NonNegativeReals)
        self.m.theta2 = Var(initialize=0.0, domain=NonNegativeReals)
        self.m.theta1.fixed = True
        self.m.theta2.fixed = True
        self.m.x1 = Var(domain=NonNegativeReals)
        self.m.x2 = Var(domain=NonNegativeReals)
        solverpath = 'C:\\w64\\glpsol'
        self.solver = SolverFactory('glpk', executable=solverpath)
        
        self.m.constraints = ConstraintList()
        constraint_list = [
            self.c1_lhs(),
            self.c2_lhs(),
            self.c3_lhs()
        ]
        for c in constraint_list:
            c_tuple = self.to_tuple(c)
            self.m.constraints.add(c_tuple)
            
        self.m.obj = Objective(expr=self.obj_expr())

        # !!! important to know if c.expr gives all constraints for indexed constraints
        # the answer is no! See notion link.
        
    def c1_lhs(self):
        return 0.8*self.m.x1 + 0.44*self.m.x2 - 24000.0 - self.m.theta1
        
    def c2_lhs(self):
        return 0.05*self.m.x1 + 0.1*self.m.x2 - 2000.0 - self.m.theta2
        
    def c3_lhs(self):
        return 0.1*self.m.x1 + 0.36*self.m.x2 - 6000.0

    def obj_expr(self):
        return - 8.1*self.m.x1 - 10.8*self.m.x2
    
    def to_tuple(self, lhs):
        return (None, lhs, 0.0)
        
    def solve(self):
        self.solver.solve(self.m, tee=True)
    
    def write(self):
        print(self.m.x1.value, self.m.x2.value)

In [1088]:
mymodel = OptiModel()
mymodel.solve()
mymodel.write()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\user1\AppData\Local\Temp\tmpibbal3oh.glpk.raw --wglp C:\Users\user1\AppData\Local\Temp\tmpahtc25sz.glpk.glp
 --cpxlp C:\Users\user1\AppData\Local\Temp\tmpdhwps_d0.pyomo.lp
Reading problem data from 'C:\Users\user1\AppData\Local\Temp\tmpdhwps_d0.pyomo.lp'...
4 rows, 3 columns, 7 non-zeros
31 lines were read
Writing problem data to 'C:\Users\user1\AppData\Local\Temp\tmpahtc25sz.glpk.glp'...
23 lines were written
GLPK Simplex Optimizer, v4.65
4 rows, 3 columns, 7 non-zeros
Preprocessing...
3 rows, 2 columns, 6 non-zeros
Scaling...
 A: min|aij| =  5.000e-02  max|aij| =  8.000e-01  ratio =  1.600e+01
GM: min|aij| =  6.252e-01  max|aij| =  1.600e+00  ratio =  2.558e+00
EQ: min|aij| =  3.909e-01  max|aij| =  1.000e+00  ratio =  2.558e+00
Constructing initial basis...
Size of triangular part is 3
*     0: obj =   0.000000000e+00 inf =   0.000e+00 (2)
*     2: obj =  -2.867586207e+05 inf =   0.000e+0

In [1089]:
for c in mymodel.m.constraints:
    print(mymodel.m.constraints[c].expr)
    
print(mymodel.m.constraints[1])

0.8*x1 + 0.44*x2 - 24000.0 - theta1  <=  0.0
0.05*x1 + 0.1*x2 - 2000.0 - theta2  <=  0.0
0.1*x1 + 0.36*x2 - 6000.0  <=  0.0
constraints[1]


In [1090]:
# fill M, m by n matrix. 
# m is no. of variables + no. of active constraints
# n is also no. of variables + no. of active constraints
# so over all it must be a square matrix.

i = 1
# print(mymodel.m.constraints[i]._body)
mymodel.m.dual.display()#
# print(list(mymodel.m.component_data_objects(Var)))
# expr.current.decompose_term((mymodel.m.constraints[i]._body))
# list(mymodel.m.dual


dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key            : Value
    constraints[1] :  -4.6551724137931
    constraints[2] : -87.5172413793103
    constraints[3] :               0.0


In [1091]:
import numpy as np

# get no of variables
var_list = list(mymodel.m.component_data_objects(Var))
no_of_var = len(var_list) -2
print('no_of_var=' + str(no_of_var)) 

# get duals as list
no_of_constr = len(mymodel.m.constraints)
print('no_of_constr=' + str(no_of_constr))

dual_list = np.zeros(shape=[no_of_constr])
# note duals are defined in opposite sign as linear program codes tend to use Ax >= b where as we use g(x) <= 0
for i in range(no_of_constr):
    dual_list[i] = - mymodel.m.dual[mymodel.m.constraints[i+1]]

print(dual_list)
active_constr_list = [mymodel.m.constraints[i+1] for i in range(0, len(dual_list)) if dual_list[i] > 0]
no_of_active_constr = len(active_constr_list)

active_dual_list = dual_list[dual_list != 0.0]


# get shape of M
M_len = no_of_var + no_of_active_constr

# Define M as 
# [ 0            0            c_x1_c1  c_x1_c2
#   .            .            c_x2_c1  c_x2_c2
#   c_x1_c1(y1)  c_x2_c1(y1)  0        0
#   c_x1_c2(y2)  c_x2_c2(y2)  0        0        ]
M = np.zeros(shape=[M_len, M_len])

# convert outer tuple to list, then inner tuples
for i in range(no_of_active_constr):
    coeff_var = expr.current.decompose_term(active_constr_list[i]._body)[1]
    coeff_var = [list(j) for j in coeff_var]

    coeff_var = np.array(coeff_var)
    coeff_var = coeff_var[:, 0].astype(float)
    
    # note below we pick 0:2 because we know the first two terms in each constraint is x1 and x2.
    # For more complicated constraints this method doesn't scale well
    M[0:no_of_var, no_of_var + i] = coeff_var.T[0:2]
    M[no_of_var + i, 0:no_of_var] = coeff_var[0:2] * active_dual_list[i]

# Define N as matrix 
N = np.zeros(shape=[M_len, 2])
N[no_of_var + 0, 0] = -active_dual_list[0]
N[no_of_var + 1, 1] = -active_dual_list[1]

y = np.linalg.solve(M, N)


no_of_var=2
no_of_constr=3
[ 4.65517241 87.51724138 -0.        ]


In [1092]:
coeff_var = expr.current.decompose_term(active_constr_list[0]._body)[1]
coeff_var

[(0.8, <pyomo.core.base.var.SimpleVar at 0x1e8ee59f588>),
 (0.44, <pyomo.core.base.var.SimpleVar at 0x1e8ee59f2e8>),
 (-24000.0, None),
 (-1, <pyomo.core.base.var.SimpleVar at 0x1e8ee59f898>)]

In [1093]:
theta0 = np.array([mymodel.m.theta1.value, mymodel.m.theta2.value])
print(theta0)
constant = np.dot(-y, -theta0.T) + np.r_[[i.value for i in var_list if i.name!='theta1' and i.name!='theta2'], active_dual_list]
slope = -y


[0. 0.]


In [1094]:
slope

array([[ 1.72413793, -7.5862069 ],
       [-0.86206897, 13.79310345],
       [-0.        , -0.        ],
       [-0.        , -0.        ]])

In [1095]:
constant

array([2.62068966e+04, 6.89655172e+03, 4.65517241e+00, 8.75172414e+01])

In [1096]:
value(slope[0][0])

1.7241379310344827

In [1097]:
mymodel.m.theta1.fixed = False
mymodel.m.theta2.fixed = False
mymodel.m.x1.fixed = True
mymodel.m.x2.fixed = True

expr_x1 = mymodel.m.theta1 * slope[0][0] + mymodel.m.theta2 * slope[0][1] + constant[0]
expr_x2 = mymodel.m.theta1 * slope[1][0] + mymodel.m.theta2 * slope[1][1] + constant[1]

# variable needs to be before its coefficient, see bug https://github.com/Pyomo/pyomo/issues/31
# mymodel.m.constraints.add(mymodel.m.x1 == mymodel.m.theta1 * slope[0][0] + mymodel.m.theta2 * slope[0][1] + constant[0])
# mymodel.m.constraints.add(mymodel.m.x2 == mymodel.m.theta1 * slope[1][0] + mymodel.m.theta2 * slope[1][1] + constant[1])
mymodel.m.constraints.add(
    0.1 * expr_x1 + 0.36 * expr_x2 - 6000.0 <= 0.0
)
# mymodel.m.constraints[1].deactivate()
# mymodel.m.constraints[2].deactivate()
mymodel.m.constraints[3].deactivate()
# mymodel.m.constraints.add(mymodel.to_tuple(mymodel.m.theta1 - 6000.0))
mymodel.m.constraints.add(mymodel.to_tuple(mymodel.m.theta2 - 500.0))
mymodel.m.obj.deactivate()
# mymodel.c3_lhs()
mymodel.m.obj2 = Objective(
    expr=0.1 * expr_x1 + 0.36 * expr_x2 - 6000.0
)

mymodel.m.obj2.sense = maximize
mymodel.m.theta1.fixed = False
mymodel.m.theta2.fixed = False
mymodel.m.x1.fixed = True
mymodel.m.x2.fixed = True

In [1098]:
mymodel.m.obj.expr

In [1099]:
mymodel.solve()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\user1\AppData\Local\Temp\tmpowbxedg5.glpk.raw --wglp C:\Users\user1\AppData\Local\Temp\tmpdoyl2upp.glpk.glp
 --cpxlp C:\Users\user1\AppData\Local\Temp\tmp55rv9grm.pyomo.lp
Reading problem data from 'C:\Users\user1\AppData\Local\Temp\tmp55rv9grm.pyomo.lp'...
5 rows, 3 columns, 6 non-zeros
34 lines were read
Writing problem data to 'C:\Users\user1\AppData\Local\Temp\tmpdoyl2upp.glpk.glp'...
25 lines were written
GLPK Simplex Optimizer, v4.65
5 rows, 3 columns, 6 non-zeros
Preprocessing...
1 row, 2 columns, 2 non-zeros
Scaling...
 A: min|aij| =  1.379e-01  max|aij| =  4.207e+00  ratio =  3.050e+01
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
*     0: obj =  -8.965517241e+02 inf =   0.000e+00 (1)
*     1: obj =  -0.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40400 bytes)
Writing basic s

In [1100]:
for c in mymodel.m.constraints:
    print(mymodel.m.constraints[c].expr)
    
value(mymodel.c3_lhs())

0.8*x1 + 0.44*x2 - 24000.0 - theta1  <=  0.0
0.05*x1 + 0.1*x2 - 2000.0 - theta2  <=  0.0
0.1*x1 + 0.36*x2 - 6000.0  <=  0.0
0.1*(1.7241379310344827*theta1 - 7.586206896551724*theta2 + 26206.8965517241) + 0.36*(-0.8620689655172413*theta1 + 13.793103448275861*theta2 + 6896.55172413793) - 6000.0  <=  0.0
theta2 - 500.0  <=  0.0


-896.5517241379348

In [1101]:
value(0.1 * expr_x1 + 0.36 * expr_x2 - 6000.0)

1.8189894035458565e-12

In [1102]:
value(
    mymodel.c3_lhs()
)

-896.5517241379348

!!!!! Note that currently when inserting values into M in the first v rows, we still need to figure out a way to insert zero for variables that don't exist (have a coeff of 0) in a constraint. <br/>
!!!!! Note that we still need a way to identify which theta belongs to which constraint.
!!!!! Note there is a challenge substituting x(theta) into the constraints. Need another generic code for that.
!!!!! Note we need to figure out a generic way to flip constraints. Probably we need to do it for subsequently added constraints. So that means these constraints, and also others, all need to come in the form of expr, inside the object. 
- Not sure how that's gonna work, probably need to have a list of expr
- This list of expr needs to be linked to the constraintlsit, so deactivations, etc. manipulations work.